In [ ]:
!pip install transformers==3.2.0

     |████████████████████████████████| 1.0MB 13.9MB/s 
     |████████████████████████████████| 1.1MB 21.5MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
     |████████████████████████████████| 3.0MB 50.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=713775f5bf2aad4eae57901d9f891bc3e20dc7e665e8c5dc78c0b23844fb6263
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import statistics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Conv1D, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Conv1D, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

nltk.download('punkt')

Mounted at /content/drive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os
import pickle
import pandas as pd

In [ ]:
path_to_file = '/content/drive/MyDrive/test_all.csv'

df = pd.read_csv(path_to_file)


In [ ]:
df.head()

,id,hyperpartisan,bias,url,labeled-by,published-at,title,body
0,17,False,left-center,http://sfgate.com/politics/article/SAN-FRANCIS...,publisher,2004-07-29,SAN FRANCISCO / Head of Juvenile Probation Dep...,Chief juvenile probation officer Gwendolyn Tu...
1,23,True,right,https://thecollegefix.com/post/32204/,publisher,2017-04-20,"University leaders ban pro-life flag display, ...",Pro-life students at Wilfrid Laurier Universit...
2,29,True,left,https://wonkette.com/622815/donald-trump-get-y...,publisher,2017-09-12,"DONALD TRUMP, GET YOUR TINY PIGGY PERVERT HAND...",Time for Hillary Clinton to go back to Benghaz...
3,32,True,right,http://thegatewaypundit.com/2016/06/dirty-croo...,publisher,2016-06-05,DIRTY: Hillary Clinton Implies Trump is a Nazi...,Do you want to know why violent unhinged lefti...
4,37,False,right-center,https://cfr.org/report/bipartisan-work-plan,publisher,2015-01-12,A Bipartisan Work Plan,American workers continue to struggle with los...


In [ ]:
df_hyper_false = df[df.hyperpartisan == False]
df_hyper_true = df[df.hyperpartisan == True]

text_values_false = df_hyper_false.body.values
text_values_true = df_hyper_true.body.values
tragets_false = df_hyper_false.hyperpartisan.values
tragets_true = df_hyper_true.hyperpartisan.values

In [ ]:
text_values_false.shape, text_values_true.shape, tragets_false.shape, tragets_true.shape

((75000,), (75000,), (75000,), (75000,))

In [ ]:
dataset_size_div_by_2 = 15_000

text_values_false = text_values_false[:dataset_size_div_by_2]
tragets_false = tragets_false[:dataset_size_div_by_2]
#############################
text_values_true = text_values_true[:dataset_size_div_by_2]
tragets_true = tragets_true[:dataset_size_div_by_2]



text = np.concatenate((text_values_false, text_values_true), axis=0)
labels = np.concatenate((tragets_false, tragets_true), axis=0)


In [ ]:
text.shape , labels.shape

((30000,), (30000,))

In [ ]:
text[0]

'Chief juvenile probation officer  Gwendolyn Tucker juvenile probation commission The Juvenile Probation Commission held an emergency meeting Monday that lasted long into the night and is reconvening again today to discuss Tucker\'s fate. Tucker, who was promoted from assistant chief last summer after then- chief  Jesse Williams Tucker has not publicly answered her accusers and referred all questions, including those about past or present gas leaks and other operational issues, to her private attorney,  Gregg Adam "She will respond to the commission," Adam said. "It\'s not appropriate for her to respond publicly." The possible shakeup at the  Juvenile Probation Department "It takes dedicated, willful leadership, the kind of leadership we might have to find outside of San Francisco," said  Andrea Shorter Center on Juvenile About a year ago, the Juvenile Probation Department adopted a new intake questionnaire meant to help probation officers determine whether recently arrested children s

In [ ]:
#####TO DO preprocess TEXT#########
text_data = []
new_labels = []
for idx, val in enumerate(text):
  if type(val) == str:
    text_data.append(val.lower())
    new_labels.append(labels[idx])
del text

In [ ]:
len(text_data)

29995

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize


In [ ]:
sent_text = []
for txt in text_data:
  sent_text.append(word_tokenize(txt))


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
def prepareBertInput(tokenizer, sentences, max_seq_length):
    input_ds = []
    attention_masks = []
    input_ds = []
    all_labels = []
    for index, sentence in enumerate(sentences):
        ids = []
        labels = []
        for word_index, word in enumerate(sentence):
            token = tokenizer.tokenize(word)
            ids.extend(token)
        if len(ids) >= max_seq_length - 2:
            ids = ids[ :(max_seq_length - 2)]
        while len(ids) < max_seq_length - 2:
            ids.append("[PAD]")
        ids.append('[SEP]')
        ids.insert(0, '[CLS]')
        ids2idx = tokenizer.convert_tokens_to_ids(ids)
        ids2idx = np.array(ids2idx)
        input_ds.append(ids2idx)
    attention_masks = []
    segments_mask = []
    for sent in input_ds:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)
    input_ds = np.array(input_ds)
    attention_masks = np.array(attention_masks)
    return [input_ds, attention_masks, np.zeros(input_ds.shape)]


In [ ]:
max_seq_length = 200
encoded = prepareBertInput(tokenizer, sent_text, max_seq_length)

In [ ]:

def getModel(LSTM_CELL_SIZE):
    in_id = tf.keras.layers.Input((max_seq_length,), dtype='int32', name="bert_input_ids")
    in_mask = tf.keras.layers.Input((max_seq_length,), dtype='int32', name="bert_input_masks")
    in_segment = tf.keras.layers.Input((max_seq_length,), dtype='int32', name="bert_segment_ids")
    inputs = [in_id, in_mask, in_segment]
    averaged_sents, _ = TFBertModel.from_pretrained("bert-base-uncased", trainable=False, from_pt=False)(inputs)
    lstm_later, forward_h, forward_c= tf.keras.layers.LSTM(LSTM_CELL_SIZE, return_sequences=True, return_state=True)(averaged_sents)
    linear =  tf.keras.layers.Dense(LSTM_CELL_SIZE )(forward_h)
    attention = tf.keras.layers.dot([lstm_later, linear], axes=(-1))
    attention = tf.keras.layers.Activation('softmax',  name='attention_vec')(attention)
    attention = tf.keras.layers.RepeatVector(LSTM_CELL_SIZE)(attention)
    attention = tf.keras.layers.Permute([2, 1])(attention)
    sent_representation = tf.keras.layers.multiply([lstm_later, attention])
    sent_representation = tf.keras.layers.Lambda(lambda xin: tf.keras.backend.sum(xin, axis=1))(sent_representation)
    sent_representation_final = tf.keras.layers.Concatenate()([sent_representation, forward_h])
    drop = tf.keras.layers.Dropout(0.2)(sent_representation)
    predictions = tf.keras.layers.Dense(2, activation='softmax')(drop)
    model = tf.keras.Model(inputs=[in_id, in_mask, in_segment] , outputs=predictions)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    return model


In [ ]:
model = getModel(200)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
vals = list(zip(encoded[0], encoded[1], encoded[2]))
X_train, X_test, y_train, y_test = train_test_split(vals, new_labels, test_size=0.1)
y_train = [to_categorical(i, num_classes=2) for i in y_train]
y_train= np.array(y_train)
y_test = [to_categorical(i, num_classes=2) for i in y_test]
y_test= np.array(y_test)

In [ ]:

X_train_ids, X_train_mask, X_train_segment = zip(*X_train)
X_train_encoded = [np.array(X_train_ids), np.array(X_train_mask), np.array(X_train_segment)]


X_test_ids, X_test_mask, X_test_segment = zip(*X_test)
X_test_encoded = [np.array(X_test_ids), np.array(X_test_mask), np.array(X_test_segment)]


In [ ]:
y_train[0]

array([1., 0.], dtype=float32)

In [ ]:
#X_train_encoded = np.array(X_train_encoded)
#X_train_encoded[0].shape

In [ ]:
print(X_train_encoded[0].shape, X_train_encoded[0].dtype)
print(X_train_encoded[1].shape, X_train_encoded[1].dtype)
print(X_train_encoded[2].shape, X_train_encoded[2].dtype)
print(y_train.shape, y_train.dtype)
print(y_test.shape, y_test.dtype)
# model.fit(X_train_encoded, y_train, shuffle=True, epochs=3, batch_size=32, validation_data=(X_test_encoded, y_test),
#          sample_weight=sample_weights)

model.fit(X_train_encoded, y_train, shuffle=True, epochs=7, batch_size=64, validation_data=(X_test_encoded, y_test))

(26995, 200) int64
(26995, 200) int64
(26995, 200) float64
(26995, 2) float32
(3000, 2) float32
Epoch 1/7
422/422 [==============================] - 467s 1s/step - loss: 0.4555 - acc: 0.7742 - val_loss: 0.3673 - val_acc: 0.8257
Epoch 2/7
422/422 [==============================] - 489s 1s/step - loss: 0.3513 - acc: 0.8403 - val_loss: 0.3222 - val_acc: 0.8550
Epoch 3/7
422/422 [==============================] - 489s 1s/step - loss: 0.3035 - acc: 0.8657 - val_loss: 0.3031 - val_acc: 0.8630
Epoch 4/7
422/422 [==============================] - 489s 1s/step - loss: 0.2633 - acc: 0.8858 - val_loss: 0.2875 - val_acc: 0.8653
Epoch 5/7
422/422 [==============================] - 489s 1s/step - loss: 0.2292 - acc: 0.9009 - val_loss: 0.3023 - val_acc: 0.8703
Epoch 6/7
422/422 [==============================] - 489s 1s/step - loss: 0.2094 - acc: 0.9122 - val_loss: 0.2966 - val_acc: 0.8663
Epoch 7/7
422/422 [==============================] - 489s 1s/step - loss: 0.1836 - acc: 0.9228 - val_loss: 0.277

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
test_pred = model.predict(X_test_encoded)


In [ ]:
pred_scores = []


In [ ]:
y_test.shape

(3000, 2)

In [ ]:
model.save_weights('/content/drive/MyDrive/Colab Notebooks/SRI_model_weights/')

In [ ]:
#42 f1 on 2 min